# Hardness properties of a polycrystal
This notebook and the accompanying code demonstrates how to use the Graph Nets library to learn to predict the hardness map of a polycrystal.

The network is trained to predict the load-depth curves of nanoindented grains in steel. 

After training, the network's prediction ability is illustrated by comparing its output to the true hardness of the material.

# Install dependencies locally

If you are running this notebook locally (i.e., not through Colaboratory), you will also need to install a few more dependencies. Run the following on the command line to install the graph networks library, as well as a few other dependencies:

```
pip install graph_nets matplotlib scipy "tensorflow>=1.15,<2" "dm-sonnet<2" "tensorflow_probability<0.9"

In [1]:
install_graph_nets_library = "No"  #param ["Yes", "No"]

if install_graph_nets_library.lower() == "yes":
  print("Installing Graph Nets library and dependencies:")
  print("Output message from command:\n")
  !pip install graph_nets "dm-sonnet<2" "tensorflow_probability<0.9"
else:
  print("Skipping installation of Graph Nets library")

Skipping installation of Graph Nets library


# load libraries

In [6]:
import configparser
confParser = configparser.ConfigParser()
confParser.read('config_irradiated.ini')
confParser.sections()


['Parameters',
 'flags',
 'gnn library path',
 'python libarary path',
 'test data files',
 'Dislocation Density']

In [7]:
# !pip install tensorflow==1.15
# import tensorflow
# tensorflow.__version__

In [8]:
import sys
sys.path.append(confParser['gnn library path']['gnnLibDir'])
sys.path.append(confParser['python libarary path']['pyLibDir'])
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pdb
import utility as utl
import warnings
warnings.filterwarnings('ignore')
import matplotlib
matplotlib.rcParams['text.usetex'] = True #--- comment tex stuff!
import os
import traceback
import imp
imp.reload(utl)
from scipy.spatial import KDTree
from scipy.interpolate import Rbf
import time
from scipy.stats import zscore

import functools
from typing import Any, Dict, Text,Tuple,Optional


import enum
import pickle
#from absl import logging
import numpy as np


import tensorflow.compat.v1 as tf
import tensorflow_probability as tfp

import sonnet as snt

from graph_nets import graphs
from graph_nets import modules as gn_modules
from graph_nets import utils_tf
from gnn_graphnet_model import *
from graph_nets import utils_np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from scipy import stats

SEED =4441666
np.random.seed(SEED)
#tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# utility funcs

In [9]:
class AddMissing:
    '''
    interpolate using k-nearest neighbors
    '''
    def __init__(self,cords,val,query_index):

        self.query_index = query_index.flatten()
        self.points    = cords
        self.val = val.copy()

    def kdTree(self,k):
        tree = KDTree( self.points )
        dd, ii = tree.query([self.points], k=k)
        self.neigh_mat= ii[0] #--- nearest neighbor matrix
        
    def interpolate(self):
        #--- nearest neighbor values
        h_mat = np.c_[list(map(lambda x:self.val[x].flatten(),self.neigh_mat))]
        assert np.all(list(map(lambda x:np.any(x>0.0),h_mat))), 'increase k!'
        #--- filter query points
        h_list = list(map(lambda x:x[x>0.0],h_mat[self.query_index]))
        assert len(h_list) == h_mat[self.query_index].shape[0]
        
        #--- average
        h_mean = list(map(lambda x: x.mean(), h_list))
        query_rows = np.arange(self.query_index.shape[0])[self.query_index]
        self.val[query_rows] = np.c_[h_mean]

        
        
def load_test_data(test_data_file_path,test_data_file_path2nd):
    # read the csv file and return the text line list.
    test_data_row_list = pd.read_csv(test_data_file_path,sep=' ')
    test_data_row_list2nd = pd.read_csv(test_data_file_path2nd,sep=' ')
    print('open and load data from test_data.csv complete.')
    return test_data_row_list, test_data_row_list2nd


def FilterDataFrame(test_data_grains,key='id',val=[1,2,3],out='index'):
    tmp = pd.DataFrame(np.c_[np.arange(test_data_grains.shape[0]),test_data_grains],columns=[out]+list(test_data_grains.keys()))
    return np.c_[tmp.set_index(key,drop=True,append=False).loc[val][out]].flatten().astype(int)

    
def base_graph( 
               test_data_file_path, 
               test_data_file_path2nd,
               predictors,
                attributes,
               logtrans=False
              ):
    """
    This here loads the data and forms a graph structure. This should be implemented as it is dataset-dependent.
    Output should be:
        a dict with  globals (dummy), nodes (nodal data in numpy), edges (edge data), receivers (indices of receiving node in int), senders (int)
        train_mask   array of size (n_nodes) with bool or 0/1 indicating training nodes
        val_mask     same for validation nodes 
        test_mask    same for testing nodes
        target 	     here the array containing the nodal target data
        weight	     if needed, a weight parameter given for the (training) nodes
    """
    
    test_data, test_data2nd = load_test_data(test_data_file_path, 
                                             test_data_file_path2nd
                                            )
    test_data_grains, test_data_grains2nd = load_test_data(test_data_file_path, 
                                         test_data_file_path2nd
                                        )

    positions = np.c_[test_data.apply(zscore)[attributes]].tolist()
    edges = np.c_[test_data2nd[['misOrientationAngle(deg)','boundaryLength(micron)']]].tolist()
    

    grain_i_indices = FilterDataFrame(test_data_grains,key='#grainID',val=test_data_grains2nd['#grain_i_ID'],out='index')
    grain_j_indices = FilterDataFrame(test_data_grains,key='#grainID',val=test_data_grains2nd['grain_j_ID'],out='index')

    receivers = list(grain_j_indices)#test_data2nd['grain_j_ID'].astype(int))
    senders = list(grain_i_indices) #test_data2nd['#grain_i_ID'].astype(int)) 
    
    #--- target vector
    target = None #list(map(lambda x:list(x),predictors))
    weight = list(np.ones(test_data.shape[0]))

    return {"globals": [0.0],  "nodes": positions, "edges": edges,  
            "receivers": receivers, "senders": senders  },target, weight 



def create_loss_ops(target_op, output_op, mask, weight=None):
    """Create supervised loss operations from targets and outputs.

    Args:
      target_op: The target tf.Tensor.
      output_ops: The output graph from the model.

    Returns:
      loss values (tf.Tensor)
    """
    if weight is None:
#         pdb.set_trace()
        loss_op = tf.reduce_mean(  (  tf.boolean_mask(output_op.nodes, mask) - tf.boolean_mask(target_op, mask))**2)
    else:
        loss_op = tf.reduce_mean( tf.boolean_mask(weight, mask)* (  tf.boolean_mask(output_op.nodes, mask) - tf.boolean_mask(target_op, mask))**2)

    return loss_op


def create_corr_ops(target_op, output_op, mask):
    corr_op = tfp.stats.correlation(tf.boolean_mask(target_op, mask), tf.boolean_mask(output_op.nodes, mask))
    return corr_op




# Creating graphs

## target data

### parse load depth curves

In [391]:
class LoadDepth:
    
    def __init__(self,loadLabels):
        self.loadLabels = loadLabels
        
    def inputFiles(self,loadLabel):
        return os.listdir('%s/%s'%(confParser['test data files']['load_depth_path'],self.loadLabels[loadLabel]))

    def indentLabels(self,loadLabel): 
        return list(map(lambda x:self.GetIndentLabel(x),self.inputFiles(loadLabel)))

    def GetIndentLabel(self,strr):
        indxi=strr.find('x')+2
        indxf=indxi+2 #strr.find('5000')
        return strr[indxi:indxf]
    
    def loadTimeSeries(self,confParser,loadLabel):
        return list(map(lambda x:pd.read_csv('%s/%s/%s'%(confParser['test data files']['load_depth_path'],self.loadLabels[loadLabel],x),
                 sep='\t',index_col=False,names=['Time','Depth','Force']),self.inputFiles(loadLabel)))

    def GetGrainId(self,indent,indentLabels,loadID,index):
        indentLabel = indentLabels[index]
        filtr = np.all([indent['label']==int(indentLabel),indent['#loadID']==loadID],axis=0)
        if not np.any(filtr):
            'warning: indenter %s not on the plane!'%indentLabel
            return indentLabel, np.nan
        grainID = indent[filtr]['grainID'].iloc[0]
        return indentLabel, grainID

    def Write(self,loadID,confParser,path_indented,path):
        loadTimeSeries = self.loadTimeSeries(confParser,loadID)
        indentLabels = self.indentLabels(loadID)
        indent = pd.read_csv('%s/indenter_grainID.txt'%path_indented,sep=' ')
        for index in range(len(loadTimeSeries)): #--- file index
            ld = loadTimeSeries[ index ] #--- file id
            indentLabel, grainID = self.GetGrainId(indent,indentLabels,loadID,index)
            print('indent label:%s, grain id:%s'%(indentLabel,grainID))
            #
            filtr = np.all([ld.Time>0],axis=0)
            np.savetxt('%s/loadDepth_GrainID_%s_LoadID%s_IndentLabel_%s.txt'%(path,grainID,loadID,int(indentLabel)),
                       np.c_[ld.Depth[filtr], ld.Force[filtr]],
                       header='Depth(nm)\tForce(mN)',
                       fmt='%4.3e\t%4.3e')

# os.system('mkdir -p %s/ldGrainID'%confParser['test data files']['load_depth_path'])
# load_depth = LoadDepth(
#        {0:'10 mN',
#         1:'7 mN',
#         2:'5 mN',
#         3:'4.5 mN',
#         4:'4 mN',
#         5:'3.5 mN',
#         6:'3 mN',
#         7:'2.5 mN',
#         8:'2 mN',
#         9:'1.75 mN',
#         10:'1.5 mN',
#         11:'1.25 mN'}
#         )
# for load_id in load_depth.loadLabels:
#     print('load_id=',load_id)
# #    load_depth.inputFiles(0)
# #    load_depth.indentLabels(0)
#     #--- read input file
# #    load_depth.loadTimeSeries(confParser,0)
      #--- write on disc
#     load_depth.Write(load_id,
#                      confParser,
#                      confParser['test data files']['ebsd_path'], #--- indented grain ids
#                      confParser['test data files']['load_depth_path']+'/ldGrainID' #--- output
#                     )


In [392]:
class Finder():
    '''
    return a list of files in a directory
    '''
    def __init__(self,path_ld):
        self.files = os.listdir(path_ld)

    def Get(self,file_index):
        return self.files[file_index]
    
class TestData:
    '''
    return the feature matrix
    '''
    ld_curve = {}
    load = {}
    
    def __init__(self,path_ld,path_gb,verbose=False):
        self.path_ld = path_ld #--- ld data
        self.path_gb = path_gb #--- grain properties
        self.verbose = verbose

    def is_included(self,xlo,xhi):
        '''
        return bool: timeseries includes the range (xlo,xhi) 
        '''
        return self.data[:,0].min() <= xlo and self.data[:,0].max() >=  xhi   
            
    def Parse(self):
        if self.verbose:
            print('parsing %s'%(self.path_ld))
        self.data    = np.loadtxt( self.path_ld ) #--- load curve
        self.grains  = pd.read_csv(self.path_gb ,sep=' ')
        
    def Interp(self,bins):
        '''
        interpolate on the structured grid bins
        '''
#        self.xsum = np.interp(bins, self.data[:,0], self.data[:,1], left=None, right=None, period=None)
#        self.edges = bins
        
        self.xsum = self.Rbf(self.data[:,1],len(bins))
        self.edges = self.Rbf(self.data[:,0],len(bins))
        
    def Rbf(self,data,nbins):
        x = np.linspace(0, 1, data.shape[0])
        d = data 
        rbfi = Rbf(x, d)
        xi = np.linspace(0, 1, nbins)
        return rbfi(xi)
            
    def GetGrainIndex(self):
        '''
        return grain index and id 
        '''
        GetGrainID = lambda x:x[x.find('ID_')+3:x.find('_LoadID')]
#         pdb.set_trace()
        GrainID = GetGrainID(self.path_ld)
        filtr = self.grains['#grainID']==float(GrainID)
        assert np.any(filtr), 'grainID %s not exist!'%GrainID
        return self.grains[filtr].index[0]

    def Scale(self):
        '''
        return scaled data 
        '''
        self.data[:,0] /= np.max(self.data[:,1]) #--- scale by fmax
        self.data[:,1] /= np.max(self.data[:,1])
        
    @staticmethod
    def ReplaceNanByMean(forces):
        mean_force_array = np.mean(forces[~np.any(np.isnan(forces),axis=1)],axis=0) #--- replace missing data by the average
        isnan = np.any(np.isnan(forces),axis=1)
        
        for grain_indx in range(len(forces)):
            if isnan[grain_indx]:
                forces[grain_indx] = np.copy(mean_force_array)


    @staticmethod
    def Append(GrainIndex,disp,load):
        '''
        append stress timeseries
        '''
        TestData.ld_curve.setdefault(GrainIndex,[]).append(disp.copy()) #--- repetative
        TestData.load.setdefault(GrainIndex,[]).append(load.copy()) #--- repetative
        
    
    @staticmethod
    def BuildFeature(sdict,ngrains_total,n_ld):
    #--- build feature matrix
        keys=list(sdict.keys()) #--- indented grains
        mat = np.c_[list(map(lambda x:np.mean(np.c_[sdict[x]],axis=0),keys))] #--- matrix
#         print(mat.shape)
        df = pd.DataFrame(np.c_[keys,mat]) #--- data frame
        df.sort_values(0,inplace=True)
#         pdb.set_trace()
        #--- non-indented grains: inser nans
    #    
        ngrains_indented = df.shape[0]
    #    n_ld = np.arange(xlo,xhi,dx).shape[0]
        ngrains = ngrains_total - ngrains_indented
        mat_nan = np.ones(ngrains*n_ld).reshape((ngrains,n_ld))*np.nan
        #
#        keys_nonindented = np.arange(ngrains_total)-keys
        keys_nonindented = set(np.arange(ngrains_total))-set(keys)
        df_nonindent = pd.DataFrame(np.c_[list(keys_nonindented),mat_nan])
        #--- combine
        mat_new = np.concatenate([df,df_nonindent],axis=0)
        df = pd.DataFrame(np.c_[mat_new]).sort_values(0,inplace=False)
    #--- row number
#        ids=list(range(ngrains_total))
#        list(map(lambda x:ids.remove(x),keys))
#        keys = keys + ids

#        df = pd.DataFrame(np.c_[keys,mat_new])
        return np.c_[df.drop(columns=[0])]

In [393]:
def main():
    !mkdir png
    symbols=utl.Symbols()

    finder=Finder(confParser['test data files']['load_depth_path']+'/ldGrainID')
    display(finder.Get(0))

    #--- parse load curve
    test_data = TestData(path_ld='%s/%s'%(confParser['test data files']['load_depth_path']+'/ldGrainID',finder.Get(1)),
                         path_gb=confParser['test data files']['test_data_file_path'],
                        )

    test_data.Parse()
    #test_data.Scale()
    GrainIndex = test_data.GetGrainIndex() #--- could be a nan!

    if not eval(confParser['flags']['remote_machine']):
        ax=utl.PltErr(test_data.data[:,0],test_data.data[:,1],
        #                xlim=(0,20),ylim=(0,0.2),
                       attrs=symbols.GetAttrs(count=0,fmt='.',nevery=2),
                      Plot=False,
                       )
        test_data.Interp(bins=np.arange(test_data.data[:,0].min(),test_data.data.max(),0.1))
        utl.PltErr(test_data.edges,test_data.xsum,
            #                xlim=(0,20),ylim=(0,0.2),
                           attrs={'fmt':'-.r'},
                            ax=ax,
                           xstr='depth(nm)',
                           ystr='load(mN)',
            #               attrs=symbols.GetAttrs(fmt='-.r'),
                           title='png/loadDepth.png'

                            )
    
#main()

### multiple grains

In [394]:
def GetDuplicateGrains():
    s=np.array(list(map(lambda x:len(TestData.ld_curve[x]),TestData.ld_curve.keys())))
    mask = s>1
    print('grains with more than one indentation data: ',np.array(list(TestData.ld_curve.keys()))[mask].shape)

def main():

    print('parse load depth curves ...')

    test_data_file_path = confParser['test data files']['test_data_file_path']
    test_data_file_path2nd = confParser['test data files']['test_data_file_path2nd']
    test_data_grains, test_data_grains2nd = load_test_data(test_data_file_path, 
                                             test_data_file_path2nd
                                            )
    ngrains = test_data_grains.shape[0]

    ax  = utl.PltErr(None,None,Plot=False)

    #--- prescribed range for displacements 
    (xlo,xhi,dx)=(0.0,1.0,0.01)

    #--- loop over indented grains
    TestData.ld_curve = {}
    TestData.load = {}
    count_indented = 0
    for fp,count in zip(finder.files,range(len(finder.files))):
        test_data = TestData(path_ld='%s/%s'%(confParser['test data files']['load_depth_path']+'/ldGrainID',fp),
                         path_gb=confParser['test data files']['test_data_file_path'],
                         verbose=False,
                        )
        test_data.Parse()
    #    test_data.Scale() #--- scale features
    #    if not test_data.is_included(xlo,xhi): #--- ld curve includes this range
    #        continue
        test_data.Interp(bins=np.arange(xlo,xhi,dx)) #--- interpolate
        try:
            GrainIndex = test_data.GetGrainIndex() #--- could be a nan!
        except:
            print('no GrainIndex for ',fp)
            continue

        if np.any(np.isnan(test_data.data[:,1])):
                print('nan in displacements: ',fp)

        #--- plot
        if not eval(confParser['flags']['remote_machine']):
            utl.PltErr(test_data.data[:,0],test_data.data[:,1],
                        attrs=symbols.GetAttrs(count=count%7),
                       ax=ax,Plot=False,
        #               xlim=(0,100),#xhi),# ylim=(0,6),
                      )
        TestData.Append(GrainIndex,test_data.edges,test_data.xsum) #--- assemble feature matrix: append displacements

        count_indented += 1
    print('# of indentation exp. is ',count_indented)

    GetDuplicateGrains()
    #--- predictors are the displacements
    predictors = TestData.BuildFeature(TestData.ld_curve, 
                                       ngrains,
                                       np.arange(xlo,xhi,dx).shape[0]
                                      )

    #---- forces
    forces = TestData.BuildFeature(TestData.load, 
                                       ngrains,
                                       np.arange(xlo,xhi,dx).shape[0]
                                      )
    TestData.ReplaceNanByMean(forces)

    print('# of indented grains: ',(~np.any(np.isnan(predictors),axis=1)).sum())

#main()

In [395]:
# filtr = test_data_grains.subBoundaryLength > 0
# test_data_grains[filtr]

In [396]:
# plt.xscale('log')
# plt.yscale('log')
# plt.scatter(test_data_grains['diameter'],test_data_grains['area'])

In [397]:
# filtr = test_data_grains.diameter > 1.0
# hist,bin_edges,err = utl.GetPDF(test_data_grains.diameter[filtr],linscale=False,n_per_decade=8) 
# #hist,bin_edges,err = utl.GetPDF(test_data_grains.diameter[filtr],linscale=True,n_per_decade=16) 
# utl.PltErr(bin_edges,hist,yerr=err,
#           xscale='log',
#            yscale='log'
#           )

In [398]:
#test_data_grains.Phi.mean(),test_data_grains.Phi.std()

### missing data

In [399]:
class AddMissing:
    '''
    interpolate using k-nearest neighbors
    '''
    def __init__(self,cords,val,query_index,verbose=False):

        self.query_index = query_index.flatten()
        self.points    = cords
        self.val = val.copy()
        self.verbose = verbose

    def kdTree(self,k):
        tree = KDTree( self.points )
        dd, ii = tree.query([self.points], k=k)
        self.neigh_mat= ii[0] #--- nearest neighbor matrix
        if self.verbose:
            print('neigh_mat.shape:',self.neigh_mat.shape)
        
    def interpolate(self):
        #--- nearest neighbor values
        h_mat = np.c_[list(map(lambda x:self.val[x],self.neigh_mat))]
        if self.verbose:
            print('h_mat.shape:',h_mat.shape)
        assert np.all(list(map(lambda x:np.any(~np.isnan(x)),h_mat))), 'increase k!'
        #--- filter query points
        h_list = list(map(lambda x:x[np.all(~np.isnan(x),axis=1)],h_mat[self.query_index]))
        assert len(h_list) == h_mat[self.query_index].shape[0]
        
        #--- average
        h_mean = list(map(lambda x: np.mean(x,axis=0), h_list))
        if self.verbose:
            print('h_mean.shape:',h_mean[0].shape)
        query_rows = np.arange(self.query_index.shape[0])[self.query_index]
        self.val[query_rows] = np.c_[h_mean]

        

# interp0 = AddMissing(np.c_[test_data_grains[['x','y']]],
#                      predictors,#np.c_[df.drop(columns=[0])],
#                      np.all(np.isnan(predictors),axis=1),
#                      verbose=True,
#                            )

# interp0.kdTree(64) #--- nearest neighbors
# interp0.interpolate()
# predictors = interp0.val.shape

## build graph

### train-test split

In [400]:
class train_test_split:
    '''
    Split arrays or matrices into random train and test subsets.
    '''
    def __init__(self,mask,
                 random_state=128,
                test_size=.3, train_size=.7):
        self.mask = mask
        self.random_state = random_state
        self.test_size=test_size
        self.train_size=train_size
        
    def train_test_split(self):
        '''
        random train-test split
        '''
        n=self.mask.shape[0]
        indented_indices = np.arange(n)[self.mask]
        n_indented = indented_indices.shape[0]
        np.random.seed(self.random_state)
        np.random.shuffle(indented_indices)
        #
        m=int(self.test_size*n_indented)
        test_set_indices = indented_indices[:m] #--- test set indices
        test_set=np.zeros(n,dtype=bool)
        test_set[test_set_indices]=True #--- assign True to test set oindices

        #
        m=int(self.train_size*n_indented)
        train_set_indices=indented_indices[n_indented-m:n_indented]
        train_set=np.zeros(n,dtype=bool)
        train_set[train_set_indices]=True #--- assign True to test set oindices

#        return train_set, test_set
        self.train_mask = train_set
        self.test_mask = test_set
    
    def train_test_split_structured(self,xy):
        '''
        structured split
        '''
        x=xy[:,0]
        y=xy[:,1]
        xlo, xhi = x.min(), x.max()
        ylo, yhi = y.min(), y.max()
        x_copy = np.array(x.copy() - xlo)
        y_copy = np.array(y.copy() - ylo)

        self.train_mask = np.all([x_copy < self.train_size * (xhi-xlo), self.mask],axis=0)
        self.test_mask = np.all([~self.train_mask, self.mask],axis=0)
    
    def train_test_split_cv(self,cv=3):
        '''
        Split arrays or matrices into *random* train and test subsets for cross validation.
        '''
        assert cv > 1, '# of partitions must be greater than 2'
        n=self.mask.shape[0]
        indented_indices = np.arange(n)[self.mask]
        n_indented = indented_indices.shape[0]
        np.random.seed(self.random_state)
        np.random.shuffle(indented_indices)
        m=int(n_indented/cv)
        test_set = {}
        for i in range(cv-1):
            test_set[i] = indented_indices[i*m:(i+1)*m] #--- test set indices
        test_set[i+1] = indented_indices[(i+1)*m:n_indented]
        assert n_indented == np.sum(list(map(lambda x:test_set[x].shape[0],test_set.keys())))
        #
        mask_dic = {}
        for i in range(cv):
            tmp_test=np.zeros(n,dtype=bool)
            tmp_test[test_set[i]]=True #--- assign True to test set oindices
            tmp_train = np.all([self.mask,~tmp_test],axis=0)
            assert not np.any(np.all([tmp_train,tmp_test],axis=0))
            mask_dic[i]={}
            mask_dic[i]['test'] = np.copy(tmp_test)
            mask_dic[i]['train'] = np.copy(tmp_train)

        self.mask_dic = mask_dic
    
    def train_test_split_cv_structured(self,xy,cv):
        '''
        Split arrays or matrices into *structured* train and test subsets for cross validation.
        '''
        m=cv[0]
        n=cv[1]
        x=xy[:,0]
        y=xy[:,1]
        xlo, xhi = x.min()-1e-6, x.max()+1e-6
        assert xhi > xlo
        ylo, yhi = y.min()-1e-6, y.max()+1e-6
        assert yhi > ylo
        x_copy = np.array(x.copy() - xlo)
        y_copy = np.array(y.copy() - ylo)
        lx = xhi-xlo
        ly = yhi-ylo
        #dy = ly / m
        #dx = lx / n
        df=pd.DataFrame(np.c_[list(map(int,m*y_copy / ly)),list(map(int,n*x_copy / lx))],columns=['row','col'])
        groups=df.groupby(by=['row','col']).groups

        mask_dic={}
        for igroup, count in zip(groups.keys(),range(m*n)):
            mask_dic[count]={}
            tmp = np.zeros(x.shape[0],dtype=bool)
            true_indices = groups[ igroup ]
            tmp[true_indices] = True
            tmp = np.all([tmp, self.mask],axis=0)
            mask_dic[count]['test'] = np.copy(tmp)
            mask_dic[count]['train'] = np.all([~tmp, self.mask],axis=0)
        #    mask_dic[count]['train'] = np.copy(tmp_train)
        self.mask_dic = mask_dic
    
    def Plot(self,test_data_grains,train_mask,test_mask):
        ax=utl.PltErr(test_data_grains[train_mask]['x'],test_data_grains[train_mask]['y'],
                  attrs={'fmt':'.'},
                 Plot=False
                 )

        utl.PltErr(test_data_grains[test_mask]['x'],test_data_grains[test_mask]['y'],
                      attrs={'fmt':'.','color':'red'},
                   ax=ax
                     )




In [401]:
def main():
    print('test/train split ...')

    mask = np.all(~np.isnan(predictors),axis=1) #--- indented grains
    #
    tsp = train_test_split(mask,
                     random_state=eval(confParser['Parameters']['seed']),
                    test_size=.3, train_size=.7)

    #--- train-test split (random)
    print('train-test split (random)')
    tsp.train_test_split()
    if not eval(confParser['flags']['remote_machine']):
        tsp.Plot(test_data_grains,
                 tsp.train_mask,
                 tsp.test_mask)

    #--- cross validate
    print('cv (random)')
    tsp.train_test_split_cv(eval(confParser['Parameters']['n_cross_val']))
    if not eval(confParser['flags']['remote_machine']):
        list(map(lambda x:tsp.Plot(test_data_grains,
                               tsp.mask_dic[x]['train'],
                               tsp.mask_dic[x]['test']),tsp.mask_dic.keys()))

    #--- train-test split (structured)
    print('train-test split (structured)')
    tsp.train_test_split_structured(np.c_[test_data_grains[['x','y']]])
    if not eval(confParser['flags']['remote_machine']):
        tsp.Plot(test_data_grains,
                 tsp.train_mask,
                 tsp.test_mask)

    #--- cross validate
    print('cv (structured)')
    n_cv = eval(confParser['Parameters']['n_cross_val'])
    tsp.train_test_split_cv_structured(np.c_[test_data_grains[['x','y']]],
                                                         (int(n_cv**.5),int(n_cv**.5))
                                                         )
    if not eval(confParser['flags']['remote_machine']):
        list(map(lambda x:tsp.Plot(test_data_grains,
                                   tsp.mask_dic[x]['train'],
                                   tsp.mask_dic[x]['test']),tsp.mask_dic.keys()))
        
#main()

In [402]:
def GetSubset(mask,fraction):
    n=mask.shape[0]
    indented_indices = np.arange(n)[mask]
    n_indented = indented_indices.shape[0]
    np.random.seed(128)
    np.random.shuffle(indented_indices)
    m=int(fraction*n_indented)
    test_set = indented_indices[:m] #--- test set indices
    #
    tmp_test=np.zeros(n,dtype=bool)
    tmp_test[test_set]=True #--- assign True to test set oindices
    return tmp_test



In [403]:
def BadGrains(mask,slist):
    for i in slist:
        mask[i]=False
#BadGrains(train_mask,[0,21,43,50,83,86,100,109,112])
#BadGrains(test_mask, [0,21,43,50,83,86,100,109,112])

### input graph

In [404]:
def main():

    #tf.set_random_seed(1234)
    #tf.get_default_graph().finalize() # something everybody tends to forget


    pure_lab='' 
    data_split = None
    data_split_lab=''
    #


    #--- path for csv data files
    test_data_file_path=confParser['test data files']['test_data_file_path']
    test_data_file_path2nd=confParser['test data files']['test_data_file_path2nd']


    
    #--- graph structure  
    static_graph_tr,\
    target_nodes_np, weight_np = base_graph(test_data_file_path, 
                                            test_data_file_path2nd,
                                            None,
                                            confParser['Parameters']['attributes'].split(), #--- attributes
                                            logtrans=data_split)
    print(static_graph_tr.keys())
    for k in static_graph_tr.keys():
        try:
            print(k, static_graph_tr[k].shape)
        except AttributeError:
            print(k)

    #--- graph without edges
    # if eval(confParser['flags']['without_edges']): 
    # #     static_graph_tr.pop('edges')
    # #     static_graph_tr.pop('receivers')
    # #     static_graph_tr.pop('senders')
    #     static_graph_tr['edges']=[static_graph_tr['edges'][0]]
    #     static_graph_tr['receivers']=[static_graph_tr['receivers'][0]]
    #     static_graph_tr['senders']=[static_graph_tr['senders'][0]]

    input_graph = utils_tf.data_dicts_to_graphs_tuple([static_graph_tr])
    graphs_tuple = utils_np.data_dicts_to_graphs_tuple([static_graph_tr])
    #print(input_graph)
    print('nodes matrix shape:',np.array(static_graph_tr['nodes']).shape)
    if not eval(confParser['flags']['without_edges']): 
        print('edge matrix shape:',np.array(static_graph_tr['edges']).shape)
        
    return static_graph_tr

static_graph_tr = main()

open and load data from test_data.csv complete.
open and load data from test_data.csv complete.
dict_keys(['globals', 'nodes', 'edges', 'receivers', 'senders'])
globals
nodes
edges
receivers
senders
nodes matrix shape: (61, 12)
edge matrix shape: (147, 2)


## Visualize 
### euclidean space

In [405]:
class interpolate:
    '''
    interpolate using k-nearest neighbors
    '''
    def __init__(self,cords,val,query_points):

        self.points    = cords
        self.query_points = query_points
        self.val = val.copy()

    def kdTree(self):
        tree = KDTree( self.points )
        dd, ii = tree.query([self.query_points], k=1)
        self.neigh_mat= ii[0] #--- nearest neighbor matrix
        
        
    def interpolate(self):        
        return self.val[self.neigh_mat.flatten()]

### nodes and edges

In [406]:
def get_cmap(n):
    colors = [(1,1,1)] + [(np.random.random(),np.random.random(),np.random.random()) for i in range(n-1)]
    new_map = matplotlib.colors.LinearSegmentedColormap.from_list('new_map', colors, N=n)
    return new_map


def main():
    if  eval(confParser['flags']['remote_machine']) or eval(confParser['flags']['without_edges']):
        return
    
    test_data_grains, test_data_grains2nd = load_test_data(confParser['test data files']['test_data_file_path'], 
                                          confParser['test data files']['test_data_file_path2nd']
                                         )

    #--- boundary segments
    data = np.loadtxt('%s/boundaryPixels.txt'%confParser['test data files']['ebsd_path'])
    df_boundary = pd.DataFrame(data,columns='grainID1 grainID2 x y phi1 Phi phi2 angle'.split())
    [xlo,xhi]=[df_boundary.x.min(),df_boundary.x.max()]
    [ylo,yhi]=[df_boundary.y.min(),df_boundary.y.max()]
    
    
    new_map = get_cmap(test_data_grains.shape[0])
    #--- plott grains
    id_matrix = np.loadtxt('%s/id_matrix.txt'%confParser['test data files']['ebsd_path'],
              )
    n=id_matrix.T.shape[1]
    val=np.array([id_matrix.T[:,(n-1)-j] for j in range(n)])
    points=np.c_[test_data_grains[['x','y']]]
#     [xlo,xhi]=[points[:,0].min(),points[:,0].max()]
#     [ylo,yhi]=[points[:,1].min(),points[:,1].max()]
    plot_bitmap = True
    if not plot_bitmap:
        val[:,:]=0
    plt.imshow(val.T,
            #            id_matrix,
                        origin='lower',
                        extent = (yhi,ylo,xlo,xhi),
            #               extent = (0,-100,-100,0),
                        cmap=new_map
                      )

    #--- plot centers
    plot_centers=True
    pixel_cutoff=0
    filtr = test_data_grains['grainSize']>pixel_cutoff
    points=np.c_[test_data_grains[filtr][['x','y']]]
    xt=points[:,0] 
    yt=points[:,1]
    ids = np.c_[test_data_grains[filtr]['#grainID']].astype(str).flatten()

    #--- plot edges
    singlePixel = test_data_grains[test_data_grains['grainSize']<=pixel_cutoff].index
    for i,j in zip(static_graph_tr['senders'],static_graph_tr['receivers']):
        if i in singlePixel or j in singlePixel:
            continue
        x=[points[i][0],points[j][0]] 
        y=[points[i][1],points[j][1]]
#         print(y,x)
        if plot_centers:
            plt.plot(y,x,
                       '-',color='black',
                     lw=.3,
                      )

    if plot_centers:
        plt.plot(yt,
                 xt,
                 '.',color='black',
                 markersize=4,
                )
#         print(ids)
        for item, text in zip(points,ids):
            plt.text(item[1],item[0],text)
        
        
    #--- boundary segments
    plt.plot(df_boundary.y,
             df_boundary.x,
             '.',color='black',
             markersize=.1,
            )
    

    #--- range
#     l=h=30
#     xc=-40;yc=-60
#     plt.xlim(xc+l/2,xc-l/2)
#     plt.ylim(yc-h/2,yc+h/2)
    plt.savefig('png/gnnMagnified.png',bbox_inches='tight',pad_inches=0.0,dpi=600)
    
main()

## Dislocation Density

In [407]:
class DislocationDensity:
    def __init__(self,id_matrix, xv, yv, **kwargs):
    
        self.id_matrix = id_matrix #--- grain-id matrix
        self.xv,self.yv = xv, yv        
        
        cart_indices = 'alpha11 alpha12 alpha13 alpha21 alpha22 alpha23 alpha31 alpha32 alpha33'.split()
        
        for key in kwargs:
            val = kwargs[ key ]
            setattr(self, key,val)
            cart_indices.remove(key) #--- set remaining indices to zero
        
        for indices in cart_indices:
            val = np.zeros(id_matrix.size).reshape(id_matrix.shape)
            key = indices
            setattr(self, key,val)

        #--- tensor norm 
        self.value = np.nansum([
            self.alpha11 * self.alpha11,
            self.alpha12 * self.alpha12,
            self.alpha13 * self.alpha13,
            self.alpha21 * self.alpha21,
            self.alpha22 * self.alpha22,
            self.alpha23 * self.alpha23,
            self.alpha31 * self.alpha31,
            self.alpha32 * self.alpha32,
            self.alpha33 * self.alpha33
                               ], axis=0) ** 0.5
        

                
    def PlotDensityMap(self,GrainID, 
                       cords,
                       boundary_segments = False,
                       grid = False,
                       **kwargs):
                
        ylo = self.yv.min()
        yhi = self.yv.max()
        xlo = self.xv.min()
        xhi = self.xv.max()
        #
        ax=utl.PltBitmap(self.value,
                   xlim = (xlo,xhi),ylim = (ylo,yhi),
                     zscore=True,
                     cmap='seismic',
                    mask = self.value == np.nan,
                     **kwargs
                  )

        #--- boundary segments
        if boundary_segments:
            x = self.cordxyBoundary.real
            y = self.cordxyBoundary.imag
            ax.plot(x,y,'.',color='brown',markersize=2)

        #--- center
#         ax.plot([cords[0]],[cords[1]],color='black',marker='.',markersize=20)

        #--- grid
        if grid:
            ax.plot(self.pixel_cords_inside.real,
                     self.pixel_cords_inside.imag,'.',color='yellow',markersize=2)

        plt.savefig(kwargs['title'],bbox_inches='tight',pad_inches=0.0)
        
    def Mask(self,GrainID):
        filtr = self.id_matrix == GrainID
        rows = np.any(filtr,axis=1)
        cols = np.any(filtr,axis=0)
        self.pixel_cords_inside = self.xv[filtr]+1j*self.yv[filtr]
        self.pixel_value_inside = self.value[filtr]

        #--- sub matrix representing the given grain
        self.id_matrix = (self.id_matrix[rows])[:,cols]
        self.value = (self.value[rows])[:,cols]
        self.xv = (self.xv[rows])[:,cols]
        self.yv = (self.yv[rows])[:,cols]

    

    def DistanceMatrix(self,center):
#         xlin = np.linspace(self.xlo,self.xhi,self.id_matrix.shape[1])
#         ylin = np.linspace(self.ylo,self.yhi,self.id_matrix.shape[0])
#         xv, yv = np.meshgrid( xlin, ylin )

#         self.rdist = (xv-center[0])+1j*(yv-center[1]) self.misOrientationBoundary
        self.nearestBoundarySeg  = np.array(list(map(lambda x:
                              np.argmin(np.abs(self.cordxyBoundary-x)),self.pixel_cords_inside)))

        self.MinDistance    = np.array(list(map(lambda x:np.abs(self.cordxyBoundary[x[1]]-x[0]),
                                                zip(self.pixel_cords_inside,self.nearestBoundarySeg))))
        
        
        self.MinOrientation = np.array(list(map(lambda x:self.misOrientationBoundary[x],
                                                self.nearestBoundarySeg)))
    
    def DensityProfile(self, indx, 
                       fout, plot=True, scale = False, **kwargs):
        rwjs = utl.ReadWriteJson(append=True)
        data = {'GrainIndex'    : indx,
                'Distance'      : self.MinDistance if not scale else  self.MinDistance / np.max( self.MinDistance ),
                'Misorientation': self.MinOrientation,
                'Density'       : self.pixel_value_inside
               }
        rwjs.Write( [ data ], fout )


#         with open(fout,'w') as fout:
#             np.savetxt(fout,np.c_[self.MinDistance/np.max(self.MinDistance),
#                                   self.pixel_value_inside],header='r rho')
        if plot:
            utl.PltErr(self.MinDistance,self.pixel_value_inside,
                       **kwargs
                      )
        
    def PrintGrainGrainDensity(self,igrain,cords,neighbors_i_indices):
        centeri = cords[ igrain ]
        
        self.rx = self.rdist.real 
        self.ry = self.rdist.imag 
        rwj = utl.ReadWriteJson()
        data = []
        for neighbor_j in neighbors_i_indices:
                    #--- neighbors ij density
            centerj = cords[ neighbor_j ]
            rij = centerj - centeri
            mag = np.sum(rij*rij)**0.5
            rij /= mag
        
            #--- project
            r_projected = self.rx * rij[ 0 ] + self.ry * rij[ 1 ]

        #--- print 
            sdict = {}
            sdict['igrain'] = float(igrain)
            sdict['jgrain'] = float(neighbor_j)
            sdict['rij'] = r_projected.flatten()
            sdict['rhoij'] = np.abs(self.value).flatten()
            data.append( sdict )
#             pdb.set_trace()
        rwj.Write(data, 'png/GrainGrainDensity_igrain%s.json'%igrain)
            
    def GrainGrainDensity(self,igrain,jgrain):
        rwj = utl.ReadWriteJson()
        try:
            idata = rwj.Read('png/GrainGrainDensity_igrain%s.json'%igrain)
            jdata = rwj.Read('png/GrainGrainDensity_igrain%s.json'%jgrain)
        except:
            print('file not found!')
            return

        for items in idata:
            assert int(items['igrain']) == igrain
            if int(items['jgrain']) == jgrain:
                rij = items['rij']
                rhoij = items['rhoij']
                break    
                
        for items in jdata:
            assert int(items['igrain']) == jgrain
            if int(items['jgrain']) == igrain:
                rji = items['rij']
                rhoji = items['rhoij']
                break
                
        #--- plot
        xdata = np.concatenate([rij-np.max(rij),np.max(rji)-rji])
        ydata = np.concatenate([np.abs(rhoij),np.abs(rhoji)])
        utl.PltErr(xdata,ydata,
           xscale='linear',yscale='log',
           attrs={'fmt':'x','alpha':1.0},
           xstr='Distance from the center (micron)', ystr=r'$|\alpha_{12}|$',
#           **kwargs
          )

    def ParseBoundarySegments(self,GrainID, data, pairs ):
        self.boundarySegments       = pd.DataFrame(data,columns='grainID1 grainID2 x y phi1 Phi phi2 angle'.split())
        filtr                       = np.any([self.boundarySegments.grainID1 == GrainID, 
                                              self.boundarySegments.grainID2 == GrainID ], axis = 0 )
        self.cordxyBoundary         = np.array(list(map(lambda x:x[0]+x[1]*1j,np.c_[self.boundarySegments[filtr]['x y'.split()]])))
#        self.misOrientationBoundary = np.c_[self.boundarySegments[filtr]['angle']]

        df                          = self.boundarySegments[ filtr ]
        filtr2nd                    = np.c_[df['grainID1 grainID2'.split()] == GrainID]
        self.neighborIdBoundary     = np.c_[df['grainID1 grainID2'.split()]][~filtr2nd]

        assert GrainID not in self.neighborIdBoundary
        assert self.neighborIdBoundary.shape[0] == self.cordxyBoundary.shape[0]
 
        
        #--- misOrientationBoundary
        neighbors = DislocationDensity.GetNeighbors(pairs,GrainID)
        self.misOrientationBoundary =\
        list(map(lambda x: neighbors[ neighbors.grain_j_ID == x]['misOrientationAngle(deg)'].iloc[ 0 ]if x != 0.0 else np.nan,
        self.neighborIdBoundary))


    @staticmethod    
    def GetNeighbors(df,df2nd,GrainID):
        filtr = np.any([df['#grain_i_ID'] == GrainID, df['grain_j_ID'] == GrainID],axis = 0)
        grain_j_IDs = list(set(list(df[filtr]['#grain_i_ID'])+list(df[filtr]['grain_j_ID'])))
        grain_j_IDs.remove(GrainID)
        return list(map(lambda x:df2nd[df2nd['#grainID']==x].index[0],grain_j_IDs))

    
    @staticmethod    
    def GetNeighbors(grain_pairs,GrainID):

        filtr = np.any([grain_pairs['#grain_i_ID'] == GrainID, grain_pairs['grain_j_ID'] == GrainID],axis=0)
        grain_pairs_filtrd = grain_pairs[ filtr ]

        ids = np.c_[grain_pairs_filtrd['#grain_i_ID grain_j_ID'.split()]]
        grain_j_IDs = ids[~(ids == GrainID)]

        return pd.DataFrame(np.c_[grain_j_IDs,grain_pairs_filtrd['misOrientationAngle(deg) boundaryLength(micron)'.split()]],
                            columns='grain_j_ID misOrientationAngle(deg) boundaryLength(micron)'.split())
    


### main()

In [ ]:
def main():

    data_grains, grain_pairs = load_test_data(
                                    confParser['test data files']['test_data_file_path'], 
                                    confParser['test data files']['test_data_file_path2nd']
                                    )
    #--- sort based on grain size 
    GrainIndices   = list(data_grains.sort_values(by='grainSize',ascending=False).index)
    
    #--- save output
    output_dir = confParser['Dislocation Density']['output_dir']
    !mkdir $output_dir
    fout = '%s/%s'%(output_dir,confParser['Dislocation Density']['fout'])
    !rm $fout

    
    for GrainIndex in GrainIndices:
        
        #--- grain id
        GrainID           =  data_grains.iloc[GrainIndex]['#grainID'].astype(int)
        print('GrainIndex=',GrainIndex, ', GrainID=',GrainID)
        
        #--- dislocation density
        cartesian_indices =  confParser['Dislocation Density']['alpha'].split() 
        #
        path              =  confParser['test data files']['ebsd_path']
        alpha             =  list(map(lambda x: np.loadtxt('%s/%s.txt'%(path,x)),cartesian_indices))
        #
        rho               =  DislocationDensity( 
                                                np.loadtxt('%s/id_matrix.txt'%path), #--- matrix of grain ids
                                                np.loadtxt('%s/x_matrix.txt'%path),  #--- xvals
                                                np.loadtxt('%s/y_matrix.txt'%path),  #--- yvals   
                                                **dict(zip(cartesian_indices,alpha))
                                              )
        #
        rho.Mask( GrainID )
        #
        rho.ParseBoundarySegments(
                                    GrainID,
                                    np.loadtxt('%s/boundaryPixels.txt'%path ),
                                    grain_pairs,
                                )
        #
        rho.PlotDensityMap(
                            GrainID, 
                            list(data_grains.iloc[GrainIndex]['x y'.split()]),
#                             grid              = True,
                            boundary_segments = True,
                            title             = '%s/densityMap_grainID%s.png'%(output_dir,GrainID),
                            colorbar          = True,
                            xlabel            = 'x(micron)',
                            ylabel            = 'y(micron)',
                            )
        #
        rho.DistanceMatrix( center = list(data_grains.iloc[GrainIndex]['x y'.split()]) )
        #        
        rho.DensityProfile( 
                            indx   =  GrainIndex,  
                            fout   =  fout,
                            plot   =  False,
                            scale  =  False,
                            xscale =  'log',
                            yscale =  'log',
#                             xlim   =  (1,10),
#                             ylim   =  (0,0.1),
                            attrs  =  {'fmt':'x','alpha':0.4},
                            xstr   =  'Min. Distance from the boundary (micron)', 
                            ystr   =  'Dislocation Density',
                            title  =  '%s/density_grainID%s.png'%(output_dir,GrainID)
                            )
#         #
#         rho.PrintGrainGrainDensity(  igrain = GrainIndex,
#                                         cords = np.c_[test_data_grains['x y'.split()]],
#                                         neighbors_i_indices = DislocationDensity.GetNeighbors(test_data_grains2nd,test_data_grains, GrainID = GrainID)
                                    
#                                     )
    return rho

rho = main()

open and load data from test_data.csv complete.
mkdir: output: File exists
GrainIndex= 47 , GrainID= 48


KeyboardInterrupt: 

In [ ]:
# dir(rho)

In [ ]:
# GrainID = 24
# indics = np.arange(rho.cordxyBoundary.shape[0])

# ib = 1111+11
# rc=(rho.cordxyBoundary.real[ib],rho.cordxyBoundary.imag[ib])#(-48,-21)
# dr=.1
# # ibs = indics[np.all([rho.cordxyBoundary.real > rc[0]-dr, 
# #                     rho.cordxyBoundary.real < rc[0]+dr, 
# #                     rho.cordxyBoundary.imag > rc[1]-dr,
# #                    rho.cordxyBoundary.imag < rc[1]+dr,
# #                    ],axis=0)]

# ax=utl.PltErr(rho.cordxyBoundary.real,rho.cordxyBoundary.imag,
#           attrs={'fmt':'s','ms':1,'color':'brown'},
#               Plot=False,
#           )

# utl.PltErr([rc[0]],[rc[1]],attrs={'fmt':'o','ms':4,'color':'red'},
#            ax=ax,
#                         Plot=False,

#           )


# if 1: #for ib in ibs:

#     x=rho.pixel_cords_inside.real #rho.pixel_cords_inside[filtr].real
#     y=rho.pixel_cords_inside.imag #[filtr].imag
    
#     utl.PltErr(x,y,attrs={'fmt':'s','ms':2,'color':'black'},Plot=False,ax=ax)
# #     utl.PltErr(rho.cordxyBoundary[ib].real,rho.cordxyBoundary[ib].imag,
# #                attrs={'fmt':'o','color':'red','ms':10},ax=ax,
              

#     filtr = rho.nearestBoundarySeg == ib
#     print('nearest points=',np.sum(filtr))
#     x=rho.pixel_cords_inside[filtr].real
#     y=rho.pixel_cords_inside[filtr].imag
    
#     utl.PltErr(x,y,attrs={'fmt':'^','ms':8,'color':'green'},Plot=False,ax=ax)

    
# #              )
# utl.PltErr(None,None,    
#               xlim=(rc[0]-3*dr,rc[0]+3*dr),
#               ylim=(rc[1]-3*dr,rc[1]+3*dr),
#            ax=ax,
#           )

In [ ]:
# print('len(rho.MinOrientation)',len(rho.MinOrientation))
# print('len(rho.misOrientationBoundary)=',len(rho.misOrientationBoundary))
# print('neighboring grain id=',rho.neighborIdBoundary[ib])
# rho.MinOrientation[filtr]
# DislocationDensity.GetNeighbors(grain_pairs,GrainID)

### AverageDensity

In [ ]:
class AverageDensity:
    
    def __init(self,**kwargs):
        
        pass
    
    def Parse( self, fp, grainIndices ):
        rwjs      = utl.ReadWriteJson()
        self.data = rwjs.Read( fp )
        self.data_concat =\
        np.concatenate(list(map(lambda x:np.c_[x['Distance'],x['Density'],x['Misorientation']]\
        if x['GrainIndex'] in grainIndices else np.c_[np.nan,np.nan,np.nan],self.data)))

    def SetFiltr( self, filtr ):
        self.filtr       =   filtr

    def Average( self, 
                LogScale = True, 
                title = 'density.png',
                Plot = False,
                **kwargs ):
        
        #--- concat data
        Distance         =   self.data_concat[ :, 0 ]
        Density          =   self.data_concat[ :, 1 ]
        Angles           =   self.data_concat[ :, 2 ]
        
        
        
        #--- preprocess

        xvals       =   Distance[ self.filtr ]
        yvals       =   Density[  self.filtr ]
            
        if xvals.shape[0] == 0:
            return

        ymax        =   np.quantile( yvals, 0.95 )
        ymin        =   np.quantile( yvals, 0.05 )

        if LogScale:
            xvals       =   np.log10( xvals )
            yvals       =   np.log10( yvals )
        
        
        #--- binning average
        xmean, ymean, _, _ = utl.GetBinnedAverage( xvals, yvals,
                                                   nbins_per_decade=4,
                                                 )
#         pdb.set_trace()
        if 'median' in kwargs and kwargs['median']:
            xmean, ymean   = AverageDensity.GetBinnedMedian( xvals, yvals,
                                                       nbins=32,
                                                     )
            
        if LogScale:
            xmean = 10**xmean
            ymean = 10**ymean
            xvals = 10**xvals
            yvals = 10**yvals
            
            
        if 'save' in kwargs:
            with open(kwargs['save'],'w') as fp:
                np.savetxt(fp,np.c_[xmean,ymean])

        #--- plot
        ax=utl.PltErr( xvals,
                       yvals,
                       attrs={'fmt':'x','alpha':1},
                       Plot=False,
                  )
        utl.PltErr( xmean, ymean,
                    attrs  =  {'fmt':'-o','color':'red',},
                    xscale =  'log' if LogScale else 'linear',
                    yscale =  'log' if LogScale else 'linear',
                    ax     =  ax,
#                     ylim   =  (ymin,ymax), #(1e-3,1e-1),#ymin,ymax),
#                     xlim   =  (0,10), #(1e-3,1e1),
                    xstr   =  'Min. Distance from the boundary (micron)', 
                    ystr   =  'Dislocation Density',
                    title  =  title,
                  )

    @staticmethod
    def GetBinnedMedian(xdata, ydata,
                         nbins=10):


        xmax = np.max(xdata) + 1.0e-10
        xmin = np.min(xdata) - 1.01e-10

        xdata_copy = np.copy(xdata)
        xdata_copy -= xmin
        xdata_copy /= (xmax - xmin)


        indices = np.array(list(map(int,xdata_copy * nbins)))

        assert np.all(indices < nbins)

        #--- group based on index
        df = pd.DataFrame(np.c_[indices,xdata,ydata],columns = 'index xvalue yvalue'.split())
        sdict = df.groupby(by='index').groups
        binnedData = np.concatenate([list(map(lambda x:df.iloc[sdict[x]].median(),sdict))],axis=1)
        return binnedData[:,1], binnedData[:,2]

    def GetGrainIndices(self,qlo,qhi, path1, path2):    
        data_grains, _ = load_test_data(
                                        path1, 
                                        path2
                                        )

        return AverageDensity.FilterGrains(data_grains,qlo=qlo, qhi=qhi) #--- below median

    @staticmethod
    def FilterGrains(data_grains,qlo,qhi):

    #     GrainIndices   = list(data_grains.sort_values(by='grainSize',ascending=False).index)
        size_qlo = np.quantile(data_grains.grainSize,qlo)
        size_qhi = np.quantile(data_grains.grainSize,qhi)
        filtr  = np.all([data_grains.grainSize >= size_qlo,
                         data_grains.grainSize < size_qhi],
                         axis=0)
        return data_grains[filtr].index


#### main()

In [ ]:
def main():

    angles = list(map(float,confParser['Dislocation Density']['angles'].split()))
    thetaLoHi  = list(zip(angles,angles[1:]))


    output_dir = confParser['Dislocation Density']['output_dir']
    finp       = '%s/%s'%(output_dir,confParser['Dislocation Density']['fout'])

    #--- lower & upper quantile in terms of size
    (qlo, qhi ) = (eval(confParser['Dislocation Density']['qlo']),\
                   eval(confParser['Dislocation Density']['qhi'])
                  )
    #--- before
    count = 0
    average_density = AverageDensity( )
    average_density.Parse(  finp,
                            average_density.GetGrainIndices(qlo,qhi,
                            confParser['test data files']['test_data_file_path'],
                            confParser['test data files']['test_data_file_path2nd']
                                                            )
                         )
    for (tlo,thi) in thetaLoHi:
        average_density.SetFiltr( np.all([  
                                            average_density.data_concat[:,1] > 0.0, 
                                            ~np.isnan(average_density.data_concat[:,2]),
                                             average_density.data_concat[:,2] >= tlo, 
                                             average_density.data_concat[:,2] < thi, 
                                         ], axis = 0 )
                                )

        average_density.Average(                        
                                LogScale = True,
                                median   = True, 
                                title    = '%s/density_angleIndx%s.png'%(output_dir,count),
                                save     = '%s/density_binAveraged_angleIndx%s.txt'%(output_dir,count),
                                )
        count += 1
main()

In [ ]:
# (edges,his,_)=plt.hist(average_density.data_concat[:,2],bins=24)
# plt.yscale('log')
# plt.ylim(1,1e6)
# his[:-1][edges>0]

#### Plot

In [ ]:
def main():
    if eval(confParser['flags']['remote_machine']):
        return
    
    legend = utl.Legends()
    legend.Set()

    symbols = utl.Symbols()
    !mkdir png
    
    Alpha = dict(zip(range(100),'alpha12 alpha13 alpha21 alpha23 alpha33'.split()))
    Qlo   = dict(zip(range(100),[0.0,0.50]))
    Qhi   = dict(zip(range(100),[0.50,1.0]))
    irradiate = dict(zip(range(100),['before','after']))
    #
#    key_alpha0 = 0
    key_q0     = 0 #--- condition based on grain size
    angleIndx  = 1 #--- condition based on misorientation of boundary segment
    #
    angles = list(map(float,confParser['Dislocation Density']['angles'].split()))
    thetaLoHi  = list(zip(angles,angles[1:]))
    output_dir = confParser['Dislocation Density']['output_dir']

    #---
    for key_alpha in Alpha:
        alpha = Alpha[key_alpha]
        for key_q in Qlo:
            qlo = Qlo[ key_q ]
            qhi = Qhi[ key_q ]
            if key_q != key_q0:
                continue
            for (tlo,thi), count in zip(thetaLoHi,range(1000)):
                if count != angleIndx:
                    continue
                ax = utl.PltErr(None,None,Plot=False)
                for key_i in irradiate:
                    str_irradiate = irradiate[ key_i ]
                    path = 'gnd/%s/q%s/alpha%s'%(str_irradiate,key_q,key_alpha)
                    try:
                        data0 = np.loadtxt('%s/Run0/%s/density_binAveraged_angleIndx%s.txt'\
                                           %(path,output_dir,count))
                    except:
                        traceback.print_exc()
                        continue


                    utl.PltErr(data0[:,0],data0[:,1],
                                ax     =  ax,
                               attrs=symbols.GetAttrs(count=key_i,label=str_irradiate,nevery=2),
                               Plot=False,
                                )
                utl.PltErr(None,None,
                            xscale =  'log',
                            yscale =  'log',
                            ax     =  ax,
                            #                         ylim   = (1e-3,1e-1),#ymin,ymax),
                            #                     xlim   =  (0,10), #(1e-3,1e1),
                            xstr   =  'Min. Distance (micron)', 
                            ystr   =  alpha, #'Dislocation Density',
                            title  =  'png/rho_alpha%s_size%s_angle%s.png'%(key_alpha,key_q,count),
                            legend =  legend.Get(),
                            dpi    =  50,
                            set_title = r'$%s^\circ~\langle~\theta~\langle~%s^\circ,%s~\langle~S~\langle~%s$'\
                                           %(int(tlo),int(thi),qlo,qhi),
                          )

main()

mkdir: png: File exists


In [ ]:
# test_data_grains, test_data_grains2nd = load_test_data(confParser['test data files']['test_data_file_path'], 
#                                       confParser['test data files']['test_data_file_path2nd']
#                                      )
# neighbors_i_indices = DislocationDensity.GetNeighbors(test_data_grains2nd,test_data_grains, GrainID = 24)
# print('neighbors of grain id=',GrainID)
# display(test_data_grains.iloc[neighbors_i_indices].sort_values(by='grainSize',ascending=False))

In [ ]:
# rho = PlotGrains( 
#     np.loadtxt('%s/id_matrix.txt'%confParser['test data files']['ebsd_path']), #--- bitmap of grain ids
#     np.loadtxt('%s/alpha12.txt'%confParser['test data files']['ebsd_path']) #--- bitmap of grain ids
#     )
# rho.GrainGrainDensity(igrain=23,
#                         jgrain=28)
